In [1]:
import numpy as np
import math

In [2]:
class KNN():
    """
    KNN-классификатор

    Параметры:
    ----------
    k: int
      Количество ближайших соседей, которое определяет
      класс предсказываемого наблюдения.

    """
    # пишем защищенный метод, вычисляющий евклидово расстояние
    def _euclidean_distance(self, x1, x2):
        """
        Вычисляет евклидово расстояние между двумя векторами
        """
        distance = 0
        for i in range(len(x1)):
            distance += pow((x1[i] - x2[i]), 2)
        return math.sqrt(distance)

    # пишем защищенный метод голосования
    def _vote(self, neighbor_labels):
        """
        Возвращает самый часто встречающийся класс
        среди ближайших соседей
        """
        # подсчитываем абсолютные частоты классов
        # для каждого наблюдения
        # возвращаем индекс максимального значения -
        # максимальной абсолютной частоты в случае классификации
        # и среднее значение - в случае регрессии
        if self.regression_mode:
            result = neighbor_labels.mean()
        else:
            counts = np.bincount(neighbor_labels.astype('int'))
            if self.cut_off is None:
                result = counts.argmax()
            else:
                result = counts.argmax() if counts.max()/counts.sum() >= self.cut_off else np.nan
        return result

    def __init__(self, k=5, task='classification', cut_off=None):
        # инициализируем k - количество ближайших соседей
        # режим classification (по-умолчанию) и regression
        # cut_off - порог для классификации, по-умолчанию None
        # и возвращается класс с максимальной частотой
        self.k = k
        self.cut_off = cut_off
        # инициализируем тип задачи
        self.regression_mode = False
        if task == 'regression':
            self.regression_mode = True
        # создаем пустой список, в котором будем
        # хранить ближайших соседей для
        # каждого наблюдения тестового набора
        self.k_nearest_neighbors_ = []

    def fit(self, X, y):
        # просто запоминаем обучающий массив признаков
        # и обучающий массив меток
        self.X_memorized = X
        self.y_memorized = y

    def predict(self, X):
        # создаем массив прогнозов, равный
        # длине тестового набора
        y_pred = np.empty(X.shape[0])
        # для каждого наблюдения тестового набора
        # предсказываем наиболее часто встречающийся
        # класс среди k ближайших соседей
        for i, test_sample in enumerate(X):
            idx = np.argsort([self._euclidean_distance(test_sample, x)
                              for x in self.X_memorized])[:self.k]
            k_nearest_neighbors = np.array([self.y_memorized[i]
                                           for i in idx])
            self.k_nearest_neighbors_.append(k_nearest_neighbors)
            y_pred[i] = self._vote(self.k_nearest_neighbors_[i])

        return y_pred

In [3]:
X_trn = np.array([[0.1, 0.2, 0.3],
                  [0.7, 0.5, 0.2],
                  [0.1, 0.2, 0.2],
                  [0.9, 0.7, 3.5],
                  [0.2, 0.4, 1.4],
                  [0.4, 0.1, 0.5]])
y_trn = np.array([1, 0, 1, 0, 0, 1])
X_tst = np.array([[0.1, 0.7, 1.1],
                  [0.5, 0.3, 2.8],
                  [0.1, 0.1, 0.2],
                  [0.9, 0.7, 1.5]])

Режим классификации

In [4]:
knn_class = KNN(k=3)
knn_class.fit(X_trn, y_trn)
pred_class = knn_class.predict(X_tst)
pred_class

array([1., 0., 1., 0.])

Режим регрессии

In [5]:
knn_regress = KNN(k=3, task='regression')
knn_regress.fit(X_trn, y_trn)
pred_regress = knn_regress.predict(X_tst)
pred_regress

array([0.66666667, 0.33333333, 1.        , 0.33333333])

In [6]:
knn_class.k_nearest_neighbors_

[array([0, 1, 1]), array([0, 0, 1]), array([1, 1, 1]), array([0, 1, 0])]

In [7]:
knn_regress.k_nearest_neighbors_

[array([0, 1, 1]), array([0, 0, 1]), array([1, 1, 1]), array([0, 1, 0])]

Посмотрим классификацию с порогом
т.к. используем эти синтетические данные, то установим порог 0.7, чтоб вошли только все единицы

In [8]:
knn_class_cut = KNN(k=3, cut_off=0.7)
knn_class_cut.fit(X_trn, y_trn)
pred_class_cut = knn_class_cut.predict(X_tst)
pred_class_cut

array([nan, nan,  1., nan])

In [9]:
knn_class_cut.k_nearest_neighbors_

[array([0, 1, 1]), array([0, 0, 1]), array([1, 1, 1]), array([0, 1, 0])]